In [ ]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

In [ ]:
import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


1.2.2
2.12.0
4.8.2
0.19.0


In [ ]:
!pip uninstall dill 
!pip install dill==0.2.8.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 KB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dill: filename=dill-0.2.8.2-py3-none-any.whl size=76466 sha256=4594758ce9f674b3f47d729a5cfeb1faae781bf5453b5c0a7af68ada4a997e50
  Stored in directory: /root/.cache/pip/wheels/6a/d4/5e/d34ef4687579ecfbf3ebfb0761292eafb69903f2ac8a717be3
Successfully built dill


In [ ]:
import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

In [ ]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/omz/new_train_del_neu_2.CSV")

In [ ]:
df

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
33759,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
33760,재미가 없으니 망하지,혐오
33761,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
33762,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [ ]:
df[ df['Sentence'].duplicated(keep=False) == True ].sort_values('Sentence')

,Sentence,Emotion
11365,7로 나온다든데 아니었나,놀람
5468,7로 나온다든데 아니었나,놀람
25643,감사합니다!,행복
25813,감사합니다!,행복
25849,감사합니다.,행복
...,...,...
6122,진짜 의외의 조합..ㅋ,놀람
27401,축하드려요~~,행복
28131,축하드려요~~,행복
25955,항상 하는 일 잘 되시고...요리 쉽게 하시는 법 알려주셔서감사합니다.,행복


In [ ]:
df["Emotion"].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
Name: Emotion, dtype: int64

In [ ]:
df = df.drop_duplicates('Sentence') 
df = df.reset_index(drop=True)

In [ ]:
emotion_mapping = {"분노":0, "놀람":1, "행복":2, "슬픔":3, "공포":4,"혐오":5 }

df['Emotion'] = df['Emotion'].map(emotion_mapping)

In [ ]:
df.value_counts(["Emotion"])

Emotion
2          6037
1          5898
0          5665
4          5468
5          5429
3          5267
dtype: int64

In [ ]:
df

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,4
1,그냥 내 느낌일뿐겠지?,4
2,아직너무초기라서 그런거죠?,4
3,유치원버스 사고 낫다던데,4
4,근데 원래이런거맞나요,4
...,...,...
33759,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,5
33760,재미가 없으니 망하지,5
33761,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,5
33762,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,5


In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df['Sentence'], df['Emotion'], 
                                                                    test_size=0.3, 
                                                                    random_state=42) 

In [ ]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [ ]:
df['Sentence'].str.len().sort_values(ascending=False)[:5]

13808    299
27433    298
33339    295
31758    261
26909    181
Name: Sentence, dtype: int64

In [ ]:
len(tokenizer.tokenize(df['Sentence'][27433]))

292

In [ ]:
df['Sentence'][27433]

'그냥 내비둬 웃음을 주잖아ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'

In [ ]:
SEQ_LEN = 300

token_ids = []
token_segments = []
token_masks = []

train_labels = []

for idx in tqdm(range(len(train_x))):

  # 'Sentence'칼럼에서 0번째 행~ 꺼내오기
  train_sentence = train_x.iloc[idx]
  print(train_sentence) 

  # 특수문자 제거
  cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", train_sentence)
  print(cleaned_sentence)

  # 토큰화 + 토큰별 시퀀스번호 부여 + 패딩
  encoded_dict = tokenizer.encode_plus(text = cleaned_sentence,
                                       padding='max_length',
                                       truncation=True,
                                       max_length=SEQ_LEN
                                       )
  
  #'input_ids', 'token_type_ids', 'attention_mask'
  token_ids.append(encoded_dict['input_ids'])
  token_masks.append(encoded_dict['attention_mask'])
  token_segments.append(encoded_dict['token_type_ids'])

  train_labels.append(train_y.iloc[idx])


train_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
train_labels = np.array(train_labels)

  0%|          | 82/23634 [00:00<00:31, 757.74it/s]

닭으로 바꿔 안죽으니까
닭으로 바꿔 안죽으니까
 총기문화 개나 줘라
 총기문화 개나 줘라
왜 반기문 노인을 들먹거려 이나라 어론들 미쳤나보지
왜 반기문 노인을 들먹거려 이나라 어론들 미쳤나보지
작작좀 해쳐먹지.
작작좀 해쳐먹지
충성심?? 개나 줘버려 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅌㅋ
충성심 개나 줘버려 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅌㅋ
이름값 하는거 보소ㄷㄷ
이름값 하는거 보소ㄷㄷ
난왜이렇게되어야했을까
난왜이렇게되어야했을까
 그것이 촛불의 힘이다.
 그것이 촛불의 힘이다
다 죽여 버릴거다
다 죽여 버릴거다
슈퍼 컴퓨터 그냥 고물상에 팔아서 엿이나 바꿔서 쳐 먹어라
슈퍼 컴퓨터 그냥 고물상에 팔아서 엿이나 바꿔서 쳐 먹어라
제약회사에서 화학과 학생이?많이 들어가나요?
제약회사에서 화학과 학생이많이 들어가나요
꿈이 소름돋을정도로 리얼했음...
꿈이 소름돋을정도로 리얼했음
난 벌금 세금 안낸다ㅋ재판해라 할 말 졸라 많으니깐..이런 개같은 나라에 단돈 일원도 못준다
난 벌금 세금 안낸다ㅋ재판해라 할 말 졸라 많으니깐이런 개같은 나라에 단돈 일원도 못준다
꿈에 왜이리 자주 나와..
꿈에 왜이리 자주 나와
솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..
솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라
대선잡고 김머중 간첩인거 밝혀내자
대선잡고 김머중 간첩인거 밝혀내자
미실(고현정)이는 저런거 없어도 날씨 잘맞춘다
미실고현정이는 저런거 없어도 날씨 잘맞춘다
><!오빠 감정 최고에요
오빠 감정 최고에요
나 참 청승맞지.
나 참 청승맞지
참 헬조선... 무전유죄유저무죄 예나 지금이나 변하지않네
참 헬조선 무전유죄유저무죄 예나 지금이나 변하지않네
누가 삼성 폭탄을 쓰냐?
누가 삼성 폭탄을 쓰냐
핵소름....??.
핵소름
유재석님 너무 축하드려요!!!
유재석님 너무 축하드려요
 이정도로만 말하면 되나요...
 이정도로만 말하면 되나요
진짜 저금은 고사하고 먹고살기가 너무 힘들고 미래가 걱정된다
진짜 저금은 고사하고 먹고살기가 너무 힘들고 미래가 걱정된

  1%|          | 259/23634 [00:00<00:28, 820.66it/s]


몸이 재산이면 감독님은 강도입니다..
몸이 재산이면 감독님은 강도입니다
그냥 평범하게 선물 사서 주기는 싫고..
그냥 평범하게 선물 사서 주기는 싫고
 감사합니다~^.~
 감사합니다
형돈오빠~~ 올만에 놀러왔어여새벽 두시가 넘었는데..ㅋㅋ
형돈오빠 올만에 놀러왔어여새벽 두시가 넘었는데ㅋㅋ
걱정되서 잠이안와요!!
걱정되서 잠이안와요
나라 망신시키는 이완용 조두순 유영철 보다 못한것이네
나라 망신시키는 이완용 조두순 유영철 보다 못한것이네
카톡안읽고 친구삭제하면 1이사라지나요
카톡안읽고 친구삭제하면 1이사라지나요
 아댓글보다 어이없어택시안에서 엄청웃었네ㅋㅋㅋㅋㅋㅋㅋ
 아댓글보다 어이없어택시안에서 엄청웃었네ㅋㅋㅋㅋㅋㅋㅋ
지구상 최악의 종자 짱께들..
지구상 최악의 종자 짱께들
잼있었어요
잼있었어요
5만리터 저장탱크도 놀랍다
5만리터 저장탱크도 놀랍다
걱정말고 하고싶은데로 해요.
걱정말고 하고싶은데로 해요
 근데 인바디 보니 체중, 체지방과 같이..
 근데 인바디 보니 체중 체지방과 같이
최순실 청와대 대장이 외교까지 진두지휘했나보네.
최순실 청와대 대장이 외교까지 진두지휘했나보네
정용기 의원이 실수로 반대방향으로들어갔군...
정용기 의원이 실수로 반대방향으로들어갔군
애들 가격은 왜 없고 청소년이냐??????? 어디에도 소인 가격 없이 장사하는 곳은 없다.
애들 가격은 왜 없고 청소년이냐 어디에도 소인 가격 없이 장사하는 곳은 없다
이세상은저를원하지않았던것처럼..
이세상은저를원하지않았던것처럼
미친 이게 보상이냐? 돌았나
미친 이게 보상이냐 돌았나
오늘 돈까스 튀겻는데 탓네요 ....
오늘 돈까스 튀겻는데 탓네요 
 어제보다 더 추운데?
 어제보다 더 추운데
결국은 멸종 아이가
결국은 멸종 아이가
오늘은 또 얼마나 맛있게 만들어 주실지 벌써부터 기대하고 있습니다
오늘은 또 얼마나 맛있게 만들어 주실지 벌써부터 기대하고 있습니다
타격이 크네..그놈의 사과폰은 왜그리 의식하노..협력업체들 어떻하냐..
타격이 크네그놈의 사과폰은 왜그리 의식하노협력업체들 어떻하냐
여자 일급

  2%|▏         | 437/23634 [00:00<00:28, 810.37it/s]

...


그때만해도 방송인데 진짜 저럴까 하고 의심할만큼 문화충격이었는데
그때만해도 방송인데 진짜 저럴까 하고 의심할만큼 문화충격이었는데
리콜후 또 터지면 어째요??
리콜후 또 터지면 어째요
아니 내가 목요일새벽부터토하고 금욜날 조퇴했거든?
아니 내가 목요일새벽부터토하고 금욜날 조퇴했거든
지금 파마중인데 비엄청온다...
지금 파마중인데 비엄청온다
검은사제들에서의 카리스마~짱!!!
검은사제들에서의 카리스마짱
박35멍충들은 썪어빠진 박근혜 똥꼬를 미친듯이 빨고있음ㅋㅋㅋㅋㅋㅋㅋㅋ
박35멍충들은 썪어빠진 박근혜 똥꼬를 미친듯이 빨고있음ㅋㅋㅋㅋㅋㅋㅋㅋ
잘되도 웃기고 잘 안되도 웃기겠다 ㅋㅋ
잘되도 웃기고 잘 안되도 웃기겠다 ㅋㅋ
장열한 최후를 맞이하셨네. 고인의명복을 빕니다.
장열한 최후를 맞이하셨네 고인의명복을 빕니다
소니가 43조라니.. 
소니가 43조라니 
??????기사거리 없어???????
기사거리 없어
피부과를 가봐야 할까요?
피부과를 가봐야 할까요
제가너무 발끈하는걸까요?.....
제가너무 발끈하는걸까요
불출석했으면서 자백강요?
불출석했으면서 자백강요
이랫다ㅡ저랫다 엠비씨 진짜 싫어 ㅎㅎㅎㅎㅎ
이랫다ㅡ저랫다 엠비씨 진짜 싫어 ㅎㅎㅎㅎㅎ
러시아 여행가보고싶은데 나같은 동ㅈ양인이 가면 살아돌아오지못할듯
러시아 여행가보고싶은데 나같은 동ㅈ양인이 가면 살아돌아오지못할듯
뭐 그럴수도있지 괜찮아 됐어!!!
뭐 그럴수도있지 괜찮아 됐어
실외인테리어도쪄죽는다이놈들아
실외인테리어도쪄죽는다이놈들아
37일 됐는데 제가 성욕이 남자친구보다 많아서ㅠㅠㅠ
37일 됐는데 제가 성욕이 남자친구보다 많아서ㅠㅠㅠ
지금 죽을것같습니다 -_
지금 죽을것같습니다 
조사해 볼 일이군...어느 부모가 아들보는 앞에서 자살할까?
조사해 볼 일이군어느 부모가 아들보는 앞에서 자살할까
이런거 꼭 기사화해야되니?
이런거 꼭 기사화해야되니
저도 즐겁습니다^^!
저도 즐겁습니다
공주인데...정부에서 보호해주는 조직이 있나보지~~
공주인데정부에서 보호해주는 조직이 있나보지
코노미웹은요?
코노미웹은요
임지봉교수님 술드

100%|█████████▉| 23577/23634 [00:38<00:00, 332.51it/s]

하... 서인영 솔비... 대박
하 서인영 솔비 대박
지금 이틀지났는데...
지금 이틀지났는데
못사는것들 ㅂㄷㅂㄷ
못사는것들 ㅂㄷㅂㄷ
최순실 박근혜보고 다 처먹어라해라ㅎ
최순실 박근혜보고 다 처먹어라해라ㅎ
서울두 비는오는데...더워요ㅠㅠ
서울두 비는오는데더워요ㅠㅠ
잊을만했는데 다시보고 아무렇지않게 지낸다면 제가 다시 그오빠를 좋아할까봐 두려워요
잊을만했는데 다시보고 아무렇지않게 지낸다면 제가 다시 그오빠를 좋아할까봐 두려워요
민주당은 김정은이 2중대.ㅋㅋㅋㅋㅋㅋㅋ
민주당은 김정은이 2중대ㅋㅋㅋㅋㅋㅋㅋ
 안드 애플os 나두고?
 안드 애플os 나두고
그리고부모님이자꾸서로가없을때저한테상대방욕을하고...
그리고부모님이자꾸서로가없을때저한테상대방욕을하고
씹선비가 저때부터 있었군
씹선비가 저때부터 있었군
롯데쌤통이다ㅋㅋㅋㅋ
롯데쌤통이다ㅋㅋㅋㅋ
뭔 뜬금포여 이게?ㅋㅋ
뭔 뜬금포여 이게ㅋㅋ
그래서 남친이 그럼 셋이 가자고 이야기를 하더라구요..
그래서 남친이 그럼 셋이 가자고 이야기를 하더라구요
어찌보면 영남대가 서강대보다 그네한테 데미지는 더 있겠네.
어찌보면 영남대가 서강대보다 그네한테 데미지는 더 있겠네
퇴진 100만 함성소리에 전율이 느껴졌다~~
퇴진 100만 함성소리에 전율이 느껴졌다
내가 로그아웃하면 끝나지?
내가 로그아웃하면 끝나지
확마 오함마로 빻아벌라
확마 오함마로 빻아벌라
신기해서 적어본다.
신기해서 적어본다
혹시 팁가지고 계신분은 알려주시면 정말 감사할게요!
혹시 팁가지고 계신분은 알려주시면 정말 감사할게요
그래서 제가 걱정되서 뭔일 잇냐고 근데 그건아니래요
그래서 제가 걱정되서 뭔일 잇냐고 근데 그건아니래요
 이제 누구한테 안기대려구요..
 이제 누구한테 안기대려구요
너무  고맙고  사랑합니다
너무  고맙고  사랑합니다
제가 옆에서 어떻게 해주면 좋을까요
제가 옆에서 어떻게 해주면 좋을까요
 인간말종이구나
 인간말종이구나
 결혼생활까지 김치년만나서 고생하고싶냐
 결혼생활까지 김치년만나서 고생하고싶냐
너무힘들어요
너무힘들어요
우리모두 준혁이성과 LG선수들

100%|██████████| 23634/23634 [00:38<00:00, 612.16it/s]

경호원은 있던데....뭐 돈 많으면 쓸 수도 있지ㅋㅋㅋ부럽다
경호원은 있던데뭐 돈 많으면 쓸 수도 있지ㅋㅋㅋ부럽다
제가 감히 생각해보건데 형의 장점은 ...우선 타격에서 그 타고난 감각으로 안타를 정말 잘 치는 것같아요...
제가 감히 생각해보건데 형의 장점은 우선 타격에서 그 타고난 감각으로 안타를 정말 잘 치는 것같아요
이러다가 우즈벡전 선발 나오는거 아냐?
이러다가 우즈벡전 선발 나오는거 아냐
 모르겟네요..ㅠㅠㅠ휴..사회적시선은 괜찮은거죠..??
 모르겟네요ㅠㅠㅠ휴사회적시선은 괜찮은거죠
퇴근까지 16분 남았당 ㅇㅅㅇ 으흐흐 공조보러가야징 넘흐 기대됨!!ㅋ
퇴근까지 16분 남았당 ㅇㅅㅇ 으흐흐 공조보러가야징 넘흐 기대됨ㅋ
 증거내놔라 숨기지말고!
 증거내놔라 숨기지말고
 그곳은 평안한지요..
 그곳은 평안한지요
하늘과 땅과 조상이  빚어낸 선녀 중에 선녀.............
하늘과 땅과 조상이  빚어낸 선녀 중에 선녀
위기를느끼고있어요..;
위기를느끼고있어요
진심으로 고마워^^
진심으로 고마워
????????? 여러 의미로 대단하네...
 여러 의미로 대단하네
우리 브로콜리 또 친정팀이라고 선물 주시네 대단하다.
우리 브로콜리 또 친정팀이라고 선물 주시네 대단하다
그래서 만져봤는데 개딱딱한거예요;;;;
그래서 만져봤는데 개딱딱한거예요
이게 므하는 짓거리지????
이게 므하는 짓거리지
제가 자기얼굴 보는거 조차 싫다는 말일까요...??
제가 자기얼굴 보는거 조차 싫다는 말일까요
정명훈을 비롯해서 이 쓰레기 같은 진실에 가담한 모두를 구속해야 한다
정명훈을 비롯해서 이 쓰레기 같은 진실에 가담한 모두를 구속해야 한다
미개한 조선놈들이었으면 농성하고 물어내라고 별 지?랄을 다 했을텐데 선진국 시민들이라 조용하네
미개한 조선놈들이었으면 농성하고 물어내라고 별 지랄을 다 했을텐데 선진국 시민들이라 조용하네
만약 탄핵부결되면 어떤일이 벌어질까
만약 탄핵부결되면 어떤일이 벌어질까
지금 생명의 위협을 느낌
지금 생명의 위협을 느낌
 대가리에 우동사리들었냐.
 대가리

In [ ]:
train_inputs

(array([[   2,  517, 5796, ...,    1,    1,    1],
        [   2, 4514, 6238, ...,    1,    1,    1],
        [   2, 3466, 2207, ...,    1,    1,    1],
        ...,
        [   2, 1701, 7636, ...,    1,    1,    1],
        [   2, 1409, 6559, ...,    1,    1,    1],
        [   2, 4102, 3220, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [ ]:
set(train_labels)

{0, 1, 2, 3, 4, 5}

In [ ]:
def create_model(max_length=100):

    bert_base_model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True) 

    input_token_ids   = layers.Input((max_length,), dtype=tf.int32, name='input_token_ids')   # tokens_tensor
    input_masks       = layers.Input((max_length,), dtype=tf.int32, name='input_masks')       # masks_tensor
    input_segments    = layers.Input((max_length,), dtype=tf.int32, name='input_segments')    # segments_tensor  

    bert_outputs = bert_base_model([input_token_ids, input_masks, input_segments]) 
    

    bert_outputs = bert_outputs[1] # ('pooler_output', )
    bert_outputs = layers.Dropout(0.2)(bert_outputs)
    final_output = layers.Dense(units=6, activation='softmax', kernel_initializer=initializers.TruncatedNormal(stddev=0.02), name="classifier")(bert_outputs)

    model = tf.keras.Model(inputs=[input_token_ids, input_masks, input_segments], 
                        outputs=final_output)

    model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5, weight_decay=0.0025, warmup_proportion=0.05),
                  loss=losses.SparseCategoricalCrossentropy(), 
                  metrics=[metrics.SparseCategoricalAccuracy()])
    
    return model

In [ ]:
# TPU setting

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("\nAll devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope(): 
    model = create_model(max_length=300)


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
# Checkpoint setting for saving the best model
from google.colab import drive
drive.mount('/gdrive')

checkpoint_path = '/gdrive/MyDrive/omz/etc/colab_data/saved_models/'

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

callback_checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_path + 'best_bert_weights.h5', 
                                                monitor='val_sparse_categorical_accuracy',
                                                save_best_only=True, 
                                                save_weights_only = True, 
                                                verbose=1) 
                                                
# Early-stopping for preventing the overfitting
callback_earlystop = callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', 
                                             min_delta=0.0001, 
                                             patience=5) 

Mounted at /gdrive


In [ ]:
history = model.fit(train_inputs, train_labels, validation_split=0.2,
                    epochs=7, batch_size=100,
                    verbose=1, 
                    callbacks=[callback_checkpoint, callback_earlystop])

Epoch 1/7
190/190 [==============================] - ETA: 0s - loss: 0.8941 - sparse_categorical_accuracy: 0.6564
Epoch 1: val_sparse_categorical_accuracy did not improve from 0.58176
190/190 [==============================] - 60s 316ms/step - loss: 0.8941 - sparse_categorical_accuracy: 0.6564 - val_loss: 1.0863 - val_sparse_categorical_accuracy: 0.5788
Epoch 2/7
190/190 [==============================] - ETA: 0s - loss: 0.8278 - sparse_categorical_accuracy: 0.6815
Epoch 2: val_sparse_categorical_accuracy improved from 0.58176 to 0.58663, saving model to /gdrive/MyDrive/omz/etc/colab_data/saved_models/best_bert_weights.h5
190/190 [==============================] - 67s 351ms/step - loss: 0.8278 - sparse_categorical_accuracy: 0.6815 - val_loss: 1.0930 - val_sparse_categorical_accuracy: 0.5866
Epoch 3/7
190/190 [==============================] - ETA: 0s - loss: 0.7707 - sparse_categorical_accuracy: 0.7048
Epoch 3: val_sparse_categorical_accuracy did not improve from 0.58663
190/190 [=====

In [ ]:
SEQ_LEN = 300 # 최대 token 개수 이상의 값으로 임의로 설정

token_ids =[]
token_masks =[]
token_segments =[]

test_labels = []


for idx in tqdm(range(len(test_x))):

    test_sentence = test_x.iloc[idx]
    
    # 특수문자 제거
    cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", test_sentence) 
    
    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=cleaned_sentence,
                                         padding='max_length', 
                                         truncation=True,
                                         max_length=SEQ_LEN)
    
    token_ids.append(encoded_dict['input_ids']) # tokens_tensor
    token_masks.append(encoded_dict['attention_mask']) # masks_tensor
    token_segments.append(encoded_dict['token_type_ids']) # segments_tensor

    test_labels.append(test_y.iloc[idx])


test_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
test_labels = np.array(test_labels)

100%|██████████| 10130/10130 [00:02<00:00, 4015.97it/s]


In [ ]:
model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5') # Load the best model's weights from checkpoint file

preds = model.predict(test_inputs)
preds = tf.argmax(preds, axis=1)

print("Accuracy: ",accuracy_score(preds, test_labels))

317/317 [==============================] - 12s 36ms/step
Accuracy:  0.5758144126357354


In [ ]:
from sklearn.metrics import precision_score , recall_score , confusion_matrix, classification_report

#target_names = ['Neutral','Positive','Negative']
target_names = ['분노','놀람','행복',"슬픔","공포","혐오"] # {"분노":0, "놀람":1, "행복":2, "슬픔":3, "공포":4,"혐오":5 }

precision = precision_score(test_labels, preds,average= "macro")
recall = recall_score(test_labels, preds,average= "macro")

print("< Confusion Matrix >\n\n",confusion_matrix(test_labels, preds))
print("\n")
print("< Classification Report >\n\n",classification_report(test_labels, preds, target_names=target_names))

< Confusion Matrix >

 [[1000  163   31   92   96  360]
 [ 144  956  148  107  273  168]
 [  23  167 1428   88   27   67]
 [  61  101   73  901  320   62]
 [  78  206   30  205 1039   63]
 [ 660  208   54  108  114  509]]


< Classification Report >

               precision    recall  f1-score   support

          분노       0.51      0.57      0.54      1742
          놀람       0.53      0.53      0.53      1796
          행복       0.81      0.79      0.80      1800
          슬픔       0.60      0.59      0.60      1518
          공포       0.56      0.64      0.60      1621
          혐오       0.41      0.31      0.35      1653

    accuracy                           0.58     10130
   macro avg       0.57      0.57      0.57     10130
weighted avg       0.57      0.58      0.57     10130



In [ ]:
from google.colab import drive
drive.mount('/gdrive')


data_path = '/gdrive/MyDrive/omz/etc/colab_data/temp_data/' 

if not os.path.exists(data_path): 
    os.makedirs(data_path)


with open(data_path + 'model_BERTfunction_v1.pkl', 'wb') as f:
    pickle.dump(dill.dumps(create_model), f) # use dill to pickle a function (https://j.mp/3CeSIzP & https://j.mp/3AaXxYW)

with open(data_path + 'tokenizer-bert.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)   

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install transformers==4.8.2
!pip install tensorflow_addons

import pandas as pd
import numpy as np

import re
import os
import pickle 
import dill # for saving a function as a file
import logging # for changing the tf's logging level
from IPython.display import clear_output 

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, initializers, losses, optimizers, metrics 

import transformers
from transformers import TFBertModel 

# 이번 실습에서 추가되었습니다
!pip install sentencepiece==0.1.96
import sentencepiece as spm 
!git clone https://github.com/monologg/KoBERT-Transformers.git 
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content 
from tokenization_kobert import KoBertTokenizer 

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

# clear the output after the installation
clear_output() 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import dill 
import pickle 

In [ ]:
!git clone https://github.com/monologg/KoBERT-Transformers.git 
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content 
from tokenization_kobert import KoBertTokenizer 

fatal: destination path 'KoBERT-Transformers' already exists and is not an empty directory.
mv: cannot stat 'KoBERT-Transformers/kobert_transformers/tokenization_kobert.py': No such file or directory


In [ ]:
# 1) Load the Model-builder function
with open("/content/drive/MyDrive/omz/etc/colab_data/temp_data/model_BERTfunction_v1.pkl", 'rb') as f: # 이번 실습에서 변경되었습니다
    create_model = pickle.load(f) # use dill to pickle a python function

In [ ]:
# 2) Load the Bert-tokenizer 
with open("/content/drive/MyDrive/omz/etc/colab_data/temp_data/tokenizer-bert.pkl", 'rb') as f:
    tokenizer = pickle.load(f) 

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=100) 

In [ ]:
checkpoint_path = '/content/drive/MyDrive/omz/etc/colab_data/saved_models/'
model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

In [ ]:
def predict_sentiment(sentence, tokenizer, model):
    
    SEQ_LEN = 100 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence),
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    prediction = model.predict(new_inputs)
    predicted_probability = np.round(np.max(prediction) * 100, 2) 
    # "분노":0, "놀람":1, "행복":2, "슬픔":3, "혐오":4,"공포":5 
    predicted_class = ["분노", "놀람", '행복',"슬픔","공포","혐오"][np.argmax(prediction, axis=1)[0]] 
    #predicted_class_2 = ["분노", "불안", '기쁨',"슬픔","당황","상처"][np.argmax(prediction, axis=1)[1]] 
    # {"분노":0, "놀람":1, "행복":2, "슬픔":3, "공포":4,"혐오":5 }
    print("첫번째 {}% 확률로 {} 리뷰입니다.".format(predicted_probability, predicted_class))
   # print("두번째 {}% 확률로 {} 리뷰입니다.".format(predicted_probability, predicted_class_2))